In [114]:
import numpy as np
import torch
import torch.nn as nn

### Попытка 1. 
Ищем только слова в середине текста, начинающиеся с большой буквы. Лучший результат – 31%, если постоянно выводить тэг PERSON.

In [196]:
lines = []
with open('input.txt') as fin:
    for line in fin.readlines():
        lines.append(line)

In [197]:
out_lines = []

In [198]:
def add_lens(split_line):
    return list(map(lambda x: (x, len(x)), split_line))

In [265]:
def delete_trash(word):
    i = 0
    prefix = ''
    word_clear = ''
    postfix = ''
    while i < len(word) and not word[i].isalpha():
        prefix += word[i]
        i += 1
    while i < len(word) and word[i].isalpha():
        word_clear += word[i]
        i += 1
    while i < len(word):
        postfix += word[i]
        i += 1
    return prefix, word_clear, postfix

In [200]:
for line in lines:
    split_line = add_lens(line.split())
    all_len = 0
    out_line = ''
    for word_token in split_line:
        word, word_len = word_token
        prefix_word, word, postfix_word = delete_trash(word)
#         print(f'pref={prefix_word} word={word} post={postfix_word}')
        if len(word) > 0 and word[0].isupper() and all_len > 0:
            index = all_len
#             tag = np.random.choice(['PERSON', 'PERSON'], p=[0.9, 0.1])
            tag = 'PERSON'
            out_line = out_line + ' ' + f'{index} {len(word)} {tag}'
        all_len += len(word) + len(prefix_word) + len(postfix_word) + 1
    out_lines.append(out_line)

In [201]:
with open('output.txt', 'a') as fout:
    for output in out_lines:
        fout.write(output + ' EOL\n')

### Попытка 2.

In [ ]:
class PersonOrOrgClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding()
        
    def forward(self, x):
        return 

In [115]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [240]:
tag_map = {0: "ORG", 1: "PERSON"}

In [254]:
def extract_tag(word, prefix):
    if '{ORG}' in prefix:
        return word, 0
    if '{PERSON}' in prefix:
        return word, 1
    return None

In [271]:
train_data = []
with open('train_sentences_enhanced.txt') as train_f:
    for line in train_f.readlines():
        train_line = []
        train_line_tags = []
        for word in line.split():
            prefix_word, word, postfix_word = delete_trash(word)
#             print(f'pref={prefix_word} word={word} post={postfix_word}')
            extract = extract_tag(word, postfix_word)
#             print(extract)
            if extract:
                train_line.append(extract[0])
                train_line_tags.append(extract[1])
        if len(train_line) > 0 and len(train_line_tags) > 0:
            train_data.append((train_line, train_line_tags))

In [272]:
train_data

[(['Скотланд', 'Руперта', 'Мердока'], [0, 1, 1]),
 (['Ванга'], [1]),
 (['Сталина', 'Вертинского'], [1, 1]),
 (['совет',
   'координаторов',
   'общественного',
   'движения',
   'Сергея',
   'Кургиняна',
   'Суть',
   'времени',
   'исполкома',
   'Единой',
   'России'],
  [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]),
 (['Холдинг', 'Газпром', 'ЗАО', 'Эхо', 'Москвы', 'радиостанции'],
  [0, 0, 0, 0, 0, 0]),
 (['ФИФА'], [0]),
 (['Яндекса', 'Mail', 'Google', 'Rambler', 'Спутник', 'Ростелекому'],
  [0, 0, 0, 0, 0, 0]),
 (['Форсайт', 'вещательной', 'корпорации', 'BBC'], [1, 0, 0, 0]),
 (['Комитета', 'по', 'иностранным', 'делам', 'Аль', 'Усамы', 'бин', 'Ладена'],
  [0, 0, 0, 0, 0, 1, 1, 1]),
 (['Марк',
   'Викиновостям',
   'Vestas',
   'Социалистической',
   'рабочей',
   'партии',
   'Британии',
   'Конгресса',
   'профсоюзов',
   'профсоюза',
   'Юнайт'],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (['Дмитрий', 'Ливанов', 'Владимира', 'Путина'], [1, 1, 1, 1]),
 (['Роснефть', 'ТНК'], [0, 0]),
 (['Лужков']

In [268]:
len(train_data)

4265

In [274]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# training_data = [
#     ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
#     ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
# ]
training_data = train_data
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {0: 0, 1: 1}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'Скотланд': 0, 'Руперта': 1, 'Мердока': 2, 'Ванга': 3, 'Сталина': 4, 'Вертинского': 5, 'совет': 6, 'координаторов': 7, 'общественного': 8, 'движения': 9, 'Сергея': 10, 'Кургиняна': 11, 'Суть': 12, 'времени': 13, 'исполкома': 14, 'Единой': 15, 'России': 16, 'Холдинг': 17, 'Газпром': 18, 'ЗАО': 19, 'Эхо': 20, 'Москвы': 21, 'радиостанции': 22, 'ФИФА': 23, 'Яндекса': 24, 'Mail': 25, 'Google': 26, 'Rambler': 27, 'Спутник': 28, 'Ростелекому': 29, 'Форсайт': 30, 'вещательной': 31, 'корпорации': 32, 'BBC': 33, 'Комитета': 34, 'по': 35, 'иностранным': 36, 'делам': 37, 'Аль': 38, 'Усамы': 39, 'бин': 40, 'Ладена': 41, 'Марк': 42, 'Викиновостям': 43, 'Vestas': 44, 'Социалистической': 45, 'рабочей': 46, 'партии': 47, 'Британии': 48, 'Конгресса': 49, 'профсоюзов': 50, 'профсоюза': 51, 'Юнайт': 52, 'Дмитрий': 53, 'Ливанов': 54, 'Владимира': 55, 'Путина': 56, 'Роснефть': 57, 'ТНК': 58, 'Лужков': 59, 'Спаситель': 60, 'Корпорация': 61, 'холдинга': 62, 'Мегафоном': 63, 'Garsdale': 64, 'Вымпелкомом': 65,

In [275]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [277]:
import tqdm

In [279]:
tqdm.tqdm_notebook(100)

0/|/| 0/? [00:00<?, ?it/s]

In [280]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in tqdm.tqdm_notebook(range(300)):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.5429, -0.8700],
        [-0.5584, -0.8490],
        [-0.5480, -0.8630]])


tensor([[-1.9036e-04, -8.5668e+00],
        [-8.4086e+00, -2.2302e-04],
        [-1.4221e+01, -7.1526e-07]])


In [283]:
model(torch.tensor(0))

TypeError: len() of a 0-d tensor